In [1]:
import pandas as pd
import statsmodels.formula.api as smf

We define a variable that contains the URL that points to the tabular data we need. We can read the csv file from the URL with pandas `read_csv()` method. It will convert csv file into a pandas DataFrame. There is a header in this csv file we got from the URL, but pandas can't process that automatically, so we will skip three rows to exclude the header by specifying "skiprows=3" in the argument.

In [2]:
ff_url = "http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_CSV.zip"
ff_factors_raw_data = pd.read_csv(ff_url, skiprows = 3)
ff_factors_raw_data

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31
...,...,...,...,...,...
1227,2017,21.51,-4.96,-13.84,0.80
1228,2018,-6.93,-3.15,-9.34,1.81
1229,2019,28.28,-6.26,-10.68,2.14
1230,2020,23.67,13.07,-47.20,0.44


We change the unnamed column with Date and Mkt-RF to Mkt_Rf. We change the name Mkt-RF because “-“ is not allowed when statsmodels interpret formula string. When defining the formula string, the column name should be a valid python identifier(I.e., the uppercase and lowercase letters A through Z, the underscore _ and, except for the first character, the digits 0 through 9.) to avoid an error or unwanted behavior when constructing a model.

In [3]:
ff_factors_raw_data.columns = ['Date', 'Mkt_RF', 'SMB', 'HML', 'RF']
ff_factors_raw_data

,Date,Mkt_RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31
...,...,...,...,...,...
1227,2017,21.51,-4.96,-13.84,0.80
1228,2018,-6.93,-3.15,-9.34,1.81
1229,2019,28.28,-6.26,-10.68,2.14
1230,2020,23.67,13.07,-47.20,0.44


We're going to use only monthly data. Since indexing with `[]` returns a view, we have to copy the result with `copy()` to avoid a warning message.

In [5]:
ff = ff_factors_raw_data[:1122].copy()
ff

,Date,Mkt_RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31
...,...,...,...,...,...
1117,201908,-2.58,-2.40,-4.85,0.16
1118,201909,1.43,-1.05,6.77,0.18
1119,201910,2.06,0.24,-1.88,0.15
1120,201911,3.87,0.91,-2.05,0.12


It's complicated and complex to handle date data as a string. Python has ‘datetime’ module that can help you manage the complexity of handling date data. So we need to change the data type of 'date' column in our DataFrame. To convert string type date to datetime object, we use pd.to_datetime() method. We pass the DataFrame or column that we want to change and the date's format as arguments. Our data has a date format as "196307," so the format token of this date is '%Y%m'. '%Y' means year and '% m' Month as a number (01-12). The common format tokens used are %Y: Year (4 digits), %m: Month, %d: Day of month, %H: Hour (24 hours), %M: Minutes, %S: Seconds. Then we set this column as an index with `set_index().`

In [6]:
ff['Date'] = pd.to_datetime(ff['Date'], format='%Y%m')
ff = ff.set_index('Date')
ff

,Mkt_RF,SMB,HML,RF
Date,,,,
1926-07-01,2.96,-2.30,-2.87,0.22
1926-08-01,2.64,-1.40,4.19,0.25
1926-09-01,0.36,-1.32,0.01,0.23
1926-10-01,-3.24,0.04,0.51,0.32
1926-11-01,2.53,-0.20,-0.35,0.31
...,...,...,...,...
2019-08-01,-2.58,-2.40,-4.85,0.16
2019-09-01,1.43,-1.05,6.77,0.18
2019-10-01,2.06,0.24,-1.88,0.15


In [9]:
ff.dtypes

Mkt_RF    object
SMB       object
HML       object
RF        object
dtype: object

'to_numeric' converts argument to a numeric type.


In [11]:
ff[['Mkt_RF', 'SMB', 'HML', 'RF']] = ff[['Mkt_RF', 'SMB', 'HML', 'RF']].apply(pd.to_numeric)
ff.dtypes

Mkt_RF    float64
SMB       float64
HML       float64
RF        float64
dtype: object

Read capm_return.csv and set the first column as the index. And specify the column that has date data with `parse_dates` parameter. So pandas can interpret the ‘Date’ as datetime object rather than just a string.

'read_csv''s index_col specifies the column index to use as the index. 
'parse_dates' will parse the date in the colummn with the index 0.

In [13]:
capm_return = pd.read_csv('capm_return.csv', index_col=0, parse_dates=[0])
capm_return

,AAPL,GSPC,IRX,AAPL_m_return,GSPC_m_return,AAPL_rp,GSPC_rp
Date,,,,,,,
2010-01-01,5.908337,1073.869995,0.070,NaN,NaN,NaN,NaN
2010-02-01,6.294721,1104.489990,0.115,0.065396,0.028514,-0.049604,-0.086486
2010-03-01,7.229299,1169.430054,0.150,0.148470,0.058796,-0.001530,-0.091204
2010-04-01,8.031907,1186.689941,0.155,0.111022,0.014759,-0.043978,-0.140241
2010-05-01,7.902396,1089.410034,0.150,-0.016125,-0.081976,-0.166125,-0.231976
...,...,...,...,...,...,...,...
2019-08-01,51.311405,2926.459961,1.933,-0.020184,-0.018092,-1.953184,-1.951092
2019-09-01,55.264339,2976.739990,1.770,0.077038,0.017181,-1.692962,-1.752819
2019-10-01,61.381241,3037.560059,1.498,0.110684,0.020432,-1.387316,-1.477568


In [14]:
capm_return = capm_return[['AAPL_m_return','GSPC_m_return','IRX','AAPL_rp','GSPC_rp']]

We're going to merge capm_return with ff. To do that, you can use merge() method from pandas. We can specify the two DataFrames you want to merge, and how you want to merge in the argument. In the argument, you can specify a type of merge to perform. There are four types of merge: left, right, outer, inner. Inner uses the intersection of keys from both frames, similar to a SQL inner join. You have to specify which key you're going to use to perform join operation from each DataFrame. One of the ways to specify them is using left_index and right_index arguments. If both are set to True, pandas will use the index column from both DataFrame as a key to perform ‘join’ operation.

In [15]:
factor_return = pd.merge(capm_return, ff, how = 'inner', left_index = True, right_index = True)
factor_return

,AAPL_m_return,GSPC_m_return,IRX,AAPL_rp,GSPC_rp,Mkt_RF,SMB,HML,RF
Date,,,,,,,,,
2010-01-01,NaN,NaN,0.070,NaN,NaN,-3.36,0.37,0.33,0.00
2010-02-01,0.065396,0.028514,0.115,-0.049604,-0.086486,3.40,1.19,3.19,0.00
2010-03-01,0.148470,0.058796,0.150,-0.001530,-0.091204,6.31,1.44,2.11,0.01
2010-04-01,0.111022,0.014759,0.155,-0.043978,-0.140241,2.00,4.86,2.91,0.01
2010-05-01,-0.016125,-0.081976,0.150,-0.166125,-0.231976,-7.89,0.14,-2.39,0.01
...,...,...,...,...,...,...,...,...,...
2019-08-01,-0.020184,-0.018092,1.933,-1.953184,-1.951092,-2.58,-2.40,-4.85,0.16
2019-09-01,0.077038,0.017181,1.770,-1.692962,-1.752819,1.43,-1.05,6.77,0.18
2019-10-01,0.110684,0.020432,1.498,-1.387316,-1.477568,2.06,0.24,-1.88,0.15


In [16]:
factor_return.dtypes

AAPL_m_return    float64
GSPC_m_return    float64
IRX              float64
AAPL_rp          float64
GSPC_rp          float64
Mkt_RF           float64
SMB              float64
HML              float64
RF               float64
dtype: object

In [17]:
factor_model = smf.ols("AAPL_m_return ~ Mkt_RF + SMB + HML", data = factor_return).fit()
factor_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          AAPL_m_return   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     16.36
Date:                Tue, 16 Mar 2021   Prob (F-statistic):           6.38e-09
Time:                        08:19:05   Log-Likelihood:                 163.07
No. Observations:                 119   AIC:                            -318.1
Df Residuals:                     115   BIC:                            -307.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0100      0.006      1.643      0.103      -0.002       0.022
Mkt_RF         0.0112      0.002      6.705      0.000       0.008       0.015
SMB           -0.0036      0.003     -1.301      0.196      -0.009       0.002
HML           -0.0063      0.003     -2.484      0.014      -0.011      -0.001
==============================================================================
Omnibus:                        9.813   Durbin-Watson:                   1.840
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               12.713
Skew:                          -0.455   Prob(JB):                      0.00174
Kurtosis:                       4.318   Cond. No.                         4.30
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [18]:
factor_return

,AAPL_m_return,GSPC_m_return,IRX,AAPL_rp,GSPC_rp,Mkt_RF,SMB,HML,RF
Date,,,,,,,,,
2010-01-01,NaN,NaN,0.070,NaN,NaN,-3.36,0.37,0.33,0.00
2010-02-01,0.065396,0.028514,0.115,-0.049604,-0.086486,3.40,1.19,3.19,0.00
2010-03-01,0.148470,0.058796,0.150,-0.001530,-0.091204,6.31,1.44,2.11,0.01
2010-04-01,0.111022,0.014759,0.155,-0.043978,-0.140241,2.00,4.86,2.91,0.01
2010-05-01,-0.016125,-0.081976,0.150,-0.166125,-0.231976,-7.89,0.14,-2.39,0.01
...,...,...,...,...,...,...,...,...,...
2019-08-01,-0.020184,-0.018092,1.933,-1.953184,-1.951092,-2.58,-2.40,-4.85,0.16
2019-09-01,0.077038,0.017181,1.770,-1.692962,-1.752819,1.43,-1.05,6.77,0.18
2019-10-01,0.110684,0.020432,1.498,-1.387316,-1.477568,2.06,0.24,-1.88,0.15


'params' will give you coefficients.

In [21]:
factor_model.params

Intercept    0.009988
Mkt_RF       0.011204
SMB         -0.003582
HML         -0.006312
dtype: float64

In [23]:
factor_exposure = factor_model.params[1:3]

In [26]:
idiosyncratic_risk = factor_model.resid.std()
idiosyncratic_risk

0.061724087740102886

In [19]:
factor_return.to_csv('factor_return.csv')